In [1]:
import pandas as pd
import numpy as np
import requests as rq
import json
import imdb
import requests
from bs4 import BeautifulSoup
import re

In [31]:
osc_bp = pd.read_csv('../data/scraping_results/osc_bp.csv')
print('Liczba filmów nominowanych do Oscara od 1927 roku: ', len(osc_bp))
osc_bp

Liczba filmów nominowanych do Oscara od 1927 roku:  591


,year,film,wiki,winner
0,1927,Wings,/wiki/Wings_(1927_film),True
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False
2,1927,The Racket,/wiki/The_Racket_(1928_film),False
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True
4,1928,Alibi,/wiki/Alibi_(1929_film),False
...,...,...,...,...
586,2022,The Fabelmans,/wiki/The_Fabelmans,False
587,2022,Tár,/wiki/T%C3%A1r,False
588,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False
589,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False


titles ma genres a ratings ma imdb rating bazy dostępne są na stronie imdb (https://developer.imdb.com/non-commercial-datasets/) za duże żeby zawrzeć je w repo

In [5]:
ratings = pd.read_csv('./ratings.tsv', delimiter='\t')
titles = pd.read_csv('./titles.tsv', delimiter='\t')

C:\Users\Aleksandra Czaplak\AppData\Local\Temp\ipykernel_9936\537975887.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('./titles.tsv', delimiter='\t')


In [6]:
print(titles.columns)
print(ratings.columns)

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'averageRating', 'numVotes'], dtype='object')


In [ ]:
ratings_df = osc_bp.merge(titles[['originalTitle', 'genres']], left_on='film', right_on='originalTitle', how='left')

use titles get genres of osc_bp

In [9]:
genres_df = osc_bp.merge(titles[['originalTitle', 'genres']], left_on='film', right_on='originalTitle', how='left')


In [11]:
genres_df.dropna(subset=['originalTitle'], inplace=True)
#filmy pojawiają się wielokrotnie z różną ilością genres - zostawiam tylko ten wiersz w którym genres jest najwięcej
genres_df['genre_count'] = genres_df['genres'].str.count(',') + 1
genres_df = genres_df.sort_values('genre_count', ascending=False).drop_duplicates('film')
genres_df.drop('genre_count', axis=1, inplace=True)
#genres_df ma 235 wierszy - osc_bp ma 591
genres_df

,year,film,wiki,winner,originalTitle,genres
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,The Broadway Melody,"Drama,Musical,Romance"
1444,1980,Raging Bull,/wiki/Raging_Bull,False,Raging Bull,"Biography,Drama,Sport"
532,1952,The Quiet Man,/wiki/The_Quiet_Man,False,The Quiet Man,"Action,Crime,Drama"
533,1953,From Here to Eternity,/wiki/From_Here_to_Eternity,True,From Here to Eternity,"Drama,Romance,War"
2074,1997,Good Will Hunting,/wiki/Good_Will_Hunting,False,Good Will Hunting,"Action,Adventure,Comedy"
...,...,...,...,...,...,...
92,1934,Viva Villa!,/wiki/Viva_Villa!,False,Viva Villa!,Western
93,1934,The White Parade,/wiki/The_White_Parade,False,The White Parade,Drama
82,1934,Flirtation Walk,/wiki/Flirtation_Walk,False,Flirtation Walk,Musical
273,1942,Kings Row,/wiki/Kings_Row,False,Kings Row,Drama


funkcja która pobiera info o genres z mojo

In [7]:


def scrape_genres(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'

    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')

    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'

    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')

    genres = []
    try:
        genre_div = box_office_soup.find('div', class_='mojo-summary-values')
        if genre_div:
            genres_element = genre_div.find('span', string='Genres')
            if genres_element:
                genres = genres_element.find_next_sibling('span').text.strip()
    except (AttributeError, TypeError):
        pass
    if genres:
        genres= re.sub(r'\n\s+', ',', genres).strip(', ')
        genres= re.sub(r'\s*,\s*', ',', genres).lower()
    print(genres, movie_title)

    return genres


In [35]:
genres_df = osc_bp.copy()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:

# Create an empty DataFrame to store the scraped results
scraped_results = pd.DataFrame(columns=['film', 'genres'])

# Iterate over each row in the DataFrame
for index, row in genres_df.iterrows():
    film_title = row['film']
    genres = scrape_genres(film_title)
    
    # Add the scraped results to the DataFrame
    scraped_results = scraped_results.append({'film': film_title, 'genres': genres}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results.to_csv('../data/additional/genres_scraped.csv', index=False)



In [26]:
genres_scraped= pd.read_csv('../data/additional/genres_scraped.csv')
genres_scraped=genres_scraped.drop('Unnamed: 0', axis=1)
genres_scraped

,film,genres
0,Wings,"action,drama,romance,war"
1,7th Heaven,drama
2,The Racket,"drama,fantasy,mystery,sci-fi,thriller"
3,The Broadway Melody,"Drama,Musical,Romance"
4,In Old Arizona,"Short,Western"
...,...,...
569,The Fabelmans,"Comedy,Talk-Show"
570,The Banshees of Inisherin,"Comedy,Drama"
571,Triangle of Sadness,"Comedy,Drama"
572,Avatar: The Way of Water,"Action,Adventure,Fantasy"


In [27]:
#filmy pojawiają się wielokrotnie z różną ilością genres - zostawiam tylko ten wiersz w którym genres jest najwięcej
genres_scraped['genre_count'] = genres_scraped['genres'].str.count(',') + 1
genres_scraped = genres_scraped.sort_values('genre_count', ascending=False).drop_duplicates('film')
genres_scraped.drop('genre_count', axis=1, inplace=True)
genres_scraped.dropna(inplace=True)
print('znaleziono genres dla', len(genres_scraped), 'filmów z', len(genres_df))

znaleziono genres dla 565 filmów z 591


dodaje kolumnę genres do genres_not_present

In [38]:
# Merge the two dataframes based on the "film" column
genres_added = genres_df.merge(genres_scraped[['film', 'genres']], on='film', how='left')

# Drop rows with NaN values in the "genres" column
genres_added = genres_added.dropna(subset=['genres'])

genres_added.to_csv('../data/additional/genres_final.csv', index=False)
# Display the updated dataframe
genres_added


,year,film,wiki,winner,genres
0,1927,Wings,/wiki/Wings_(1927_film),True,"action,drama,romance,war"
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,drama
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,"drama,fantasy,mystery,sci-fi,thriller"
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,"Drama,Musical,Romance"
4,1928,Alibi,/wiki/Alibi_(1929_film),False,"crime,thriller"
...,...,...,...,...,...
586,2022,The Fabelmans,/wiki/The_Fabelmans,False,"Comedy,Talk-Show"
587,2022,Tár,/wiki/T%C3%A1r,False,"Drama,Music"
588,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,"Comedy,Talk-Show"
589,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"Comedy,Drama"


RATING

In [39]:
import imdb

def get_imdb_scores(movie_title):
    """
    Given a movie title, returns a dictionary containing the IMDb score and number of votes for the movie.
    """
    ia = imdb.IMDb()
    scores = {}
    # search for the movie by title
    results = ia.search_movie(movie_title)
    if results:
        # use the first search result to retrieve the IMDb score and number of votes
        movie = results[0]
        ia.update(movie)
        score = movie.get('rating')
        votes = movie.get('votes')
        scores['score'] = score
        scores['votes'] = votes
    else:
        # if no search results are found, add entries with None values
        scores['score'] = None
        scores['votes'] = None
    return scores


In [45]:
rating_df = osc_bp.copy()

In [ ]:

scraped_results_votes = pd.DataFrame(columns=['film', 'rating', 'numVotes'])
# Iterate over each row in the DataFrame
for index, row in rating_df.iterrows():
    film_title = row['film']
    rating = get_imdb_scores(film_title)['score']
    numVotes = get_imdb_scores(film_title)['votes']
    
    # Add the scraped results to the DataFrame
    scraped_results_votes  = scraped_results_votes.append({'film': film_title, 'rating': rating, 'numVotes':numVotes}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results_votes.to_csv('../data/additional/ratings_scraped.csv', index=False)




In [48]:

ratings_scraped = pd.read_csv('../data/additional/ratings_scraped.csv')
rating_added = rating_df.merge(ratings_scraped[['film', 'rating', 'numVotes']], on='film', how='left')
# Drop rows with NaN values in the "ratings" column
rating_added = rating_added.dropna(subset=['rating']).drop_duplicates('wiki')
print('znaleziono rating dla ', len(rating_added), ' z ', len(osc_bp), 'filmów')
rating_added.to_csv('../data/additional/ratings_final.csv')

znaleziono rating dla  583  z  591 filmów


now adding box office

funkcja która pobiera z mojo info o box office

In [38]:
import requests
from bs4 import BeautifulSoup

def scrape_box_office_info(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'
    
    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    
    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        print(f'No box office information found for {movie_title}')
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        print(f'No box office information found for {movie_title}')
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'
    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')
   

    box_office = {'Domestic': 'N/A', 'International': 'N/A', 'Worldwide': 'N/A'}
    try:
        table = box_office_soup.find('div', class_='a-section a-spacing-none mojo-performance-summary-table')
        rows = table.find_all('div', class_='a-section a-spacing-none')
        for row in rows:
            span = row.find('span', class_='a-size-small')
            if span:
                title = span.text.strip()
                money = row.find('span', class_='money')
                if money:
                    box_office[title.split()[0]] = money.text.strip()
                else:
                    box_office[title.split()[0]] = 'N/A'
    except (AttributeError, TypeError):
        pass
    print(box_office, movie_title)
    
    return box_office

In [ ]:
bo_df = osc_bp.copy()
# Apply the function to each row in the "film" column
bo_df['box_office'] = bo_df['film'].apply(scrape_box_office_info)



In [99]:
#Usunięcie NaN 
bo_df = bo_df.dropna(subset=['box_office'])
# Sprawdzenie liczby duplikatów w kolumnie 'film'
num_duplicates = bo_df['film'].duplicated().sum()
bo_df = bo_df.drop_duplicates(subset=['film'])
# Wyświetlenie liczby duplikatów
print("Usunięto: ", num_duplicates, ' duplikatów')

In [112]:
bo_df.to_csv('../data/additional/bo_scraped.csv')

In [54]:
bo_df = pd.read_csv('../data/additional/bo_scraped.csv')
bo_df = bo_df.drop('Unnamed: 0', axis=1)
bo_df

,year,film,wiki,winner,box_office
0,1927,Wings,/wiki/Wings_(1927_film),True,"{'Domestic': 'N/A', 'International': '$746', '..."
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,"{'Domestic': '$79,808', 'International': 'N/A'..."
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,"{'Domestic': '$6,303,762', 'International': '$..."
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,"{'Domestic': 'N/A', 'International': '$223,723..."
4,1928,Alibi,/wiki/Alibi_(1929_film),False,"{'Domestic': '$24,845', 'International': '$18,..."
...,...,...,...,...,...
554,2022,Tár,/wiki/T%C3%A1r,False,"{'Domestic': '$6,773,650', 'International': '$..."
555,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,"{'Domestic': '$718,732,821', 'International': ..."
556,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"{'Domestic': '$4,608,096', 'International': '$..."
557,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,True,"{'Domestic': '$77,191,785', 'International': '..."


In [56]:
import ast

# Definicja funkcji sprawdzającej strukturę wartości box_office
def check_structure(value):
    expected_structure = {'Domestic': None, 'International': None, 'Worldwide': None}
    return isinstance(value, dict) and value.keys() == expected_structure.keys()

# Convert string values to dictionaries
def parse_box_office(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value.replace("'", "\""))
        except (ValueError, SyntaxError):
            return None
    return value

bo_cleaned = bo_df.copy()

bo_cleaned['box_office'] = bo_cleaned['box_office'].apply(parse_box_office)


# Wyświetlenie wierszy, w których struktura wartości box_office jest inna niż oczekiwana
mismatched_rows = bo_cleaned[~bo_cleaned['box_office'].apply(check_structure)]

# Wyświetlenie wyniku
print('obecnie ', len(mismatched_rows), 'posiada niepoprawną strukturę danych')


obecnie  0 posiada niepoprawną strukturę danych


In [61]:
bo_final=bo_cleaned.copy()
# Create separate columns for each box office category
bo_final[['domestic_box_office', 'international_box_office', 'worldwide_box_office']] = bo_final['box_office'].apply(pd.Series)
bo_final = bo_final.drop(['box_office', 'domestic_box_office', 'international_box_office'], axis=1)
bo_final = bo_final.dropna(subset=['worldwide_box_office'])
print('znaleziono rating dla ', len(bo_final), ' z ', len(osc_bp), 'filmów')
bo_final

znaleziono rating dla  559  z  591 filmów


,year,film,wiki,winner,worldwide_box_office
0,1927,Wings,/wiki/Wings_(1927_film),True,$746
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,"$79,808"
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,"$21,733,230"
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,"$223,723"
4,1928,Alibi,/wiki/Alibi_(1929_film),False,"$42,915"
...,...,...,...,...,...
554,2022,Tár,/wiki/T%C3%A1r,False,"$27,541,681"
555,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,"$1,493,491,858"
556,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"$25,615,870"
557,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,True,"$140,211,159"


In [62]:
bo_final.to_csv('../data/additional/bo_final.csv')